In [4]:
import numpy as np
import pandas as pd
import polars as pl

In [13]:
df = pd.read_csv('variant_v.csv')
df1 = pd.read_csv('base.csv')

In [51]:
df_pl = pl.read_csv("variant_v.csv")


In [65]:
def ingest(path,has_header=True, columns=None):
    '''
    Reads a file into a Dataframe
    
    path: Path to a file or a file-like object.
    By file-like object, we refer to objects with a ``read()``
    method
    
    has_header: Indicate if the first row of dataset is a header or not.
    
    '''
    try:
        df = pl.read_csv(path,has_header,columns)
        shape = df.shape
        if shape[0] == 0:
            print("Empty Dataframe with (0,0) entries")
        else:
            return df
            
    except FileNotFoundError:
        print("No such file or directory: " + "{}".format(path))
        
def metric_summary(df, path_to_save=None):
    
    

In [75]:
df = ingest("variant_v.csv")
df

AttributeError: 'DataFrame' object has no attribute 'summary'